# Intro / Business Problem

#### Around five million people die every year — almost a third of them in India— due to inadequate healthcare (Times of India). Your organization has raised significant funds to open a free health clinic in Mumbai, the largest city in India, to help reduce deaths that could be avoided with proper healthcare. Mumbai is a huge city, but you currently only have funding to open one free clinic. Which neighborhood of Mumbai most needs this assistance?

# Data

#### To answer this question, I will use the data from two different sources:

#### The first is geolocation data website Foursquare, which will allow me to see where all the clinics and hospitals are located in Mumbai. 

#### The second is 99 Acres, an Indian real estate database website. I will look at property values across the neighborhoods of Mumbai, and then I will assume that average property value and average income are directly related. Found here: https://www.99acres.com/property-rates-and-price-trends-in-mumbai

#### I will then cluster neighborhoods by average income and healthcare venues. I will suggest neighborhoods with lowest income and least healthcare options to be the location for the free health clinic.

# Methodology

 <span style='color:Blue'> 
    1) Import libraries and download dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # client for geocoding webservices
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans # machine learning method for splitting data into like groups

!conda install -c conda-forge folium=0.5.0 --yes # install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

 <span style='color:Blue'>
    2) Import the property value dataset from 99acres and clean dataframe (https://www.99acres.com/property-rates-and-price-trends-in-mumbai)

In [2]:
mumbaipropertyvalue=pd.read_csv("https://raw.githubusercontent.com/mkatestephens/Coursera_Capstone/master/Mumbai_Property_Values.csv")
mumbaipropertyvalue[['locality name','Price Range']].dropna().head(10)

,locality name,Price Range
0,Anand Nagar,"Rs. 7,735 - 8,628/sq. ft."
1,Ashok Nagar,"Rs. 8,840 - 10,242/sq. ft."
2,Balkum,"Rs. 8,755 - 9,988/sq. ft."
3,Balkum Pada,"Rs. 8,542 - 9,095/sq. ft."
4,Bhaskar Colony,"Rs. 13,940 - 15,555/sq. ft."
5,Bhayandarpada,"Rs. 7,012 - 7,905/sq. ft."
6,Brahmand,"Rs. 8,415 - 9,308/sq. ft."
7,Budhaji Nagar,"Rs. 7,268 - 8,202/sq. ft."
8,Charai,"Rs. 10,158 - 12,155/sq. ft."
9,Chirak Nagar,"Rs. 10,455 - 11,688/sq. ft."


In [3]:
# Assign PIN codes (Indian postal codes) to each neighborhood

url="https://nominatim.openstreetmap.org/search/Andheri%20Mumbai?format=json&addressdetails=1&limit=1&polygon_svg=1"
result=requests.get(url).json()
result[0]['lat']

'19.1196976'

In [4]:
url="https://nominatim.openstreetmap.org/search/{}%20{}?format=json&addressdetails=1&limit=1&polygon_svg=1".format("4 Bungalows","Mumbai")
result=requests.get(url).json()
mumbaipropertyvalue['Longitude']=None

In [5]:
#Assign Latitude and longitude to each neighborhood

for index, row in mumbaipropertyvalue.iterrows():

    locality=row['locality name']

#initialize your variable to None 


#loop until you get the coordinates

    #while(lat_lng_coords is None):
    url="https://nominatim.openstreetmap.org/search/{}%20{}?format=json&addressdetails=1&limit=1&polygon_svg=1".format(locality,"Mumbai")
    result=requests.get(url).json()
    if result!=[]:
        mumbaipropertyvalue.loc[index,'Latitude']=result[0]['lat']
        mumbaipropertyvalue.loc[index,'Longitude']=result[0]['lon']

    else:
        mumbaipropertyvalue.loc[index,'Latitude']=None
        mumbaipropertyvalue.loc[index,'Longitude']=None

In [6]:
# Make sure latitude and longitude added correctly
mumbaipropertyvalue.head()

,locality name,Price Range,Q/Q,trends,one bedroom,two bedroom,three bedroom,Longitude,Latitude
0,Anand Nagar,"Rs. 7,735 - 8,628/sq. ft.",-1.03%,See Trends,"Rs. 10,566 - 12,486","Rs. 15,300 - 18,360","Rs. 19,890 - 24,310",72.8118877,18.9665234
1,Ashok Nagar,"Rs. 8,840 - 10,242/sq. ft.",-,Not Available,-,"Rs. 18,553 - 20,973",-,72.8797875821267,19.0529955
2,Balkum,"Rs. 8,755 - 9,988/sq. ft.",-2.24%,See Trends,"Rs. 12,928 - 15,415","Rs. 17,850 - 20,825","Rs. 25,517 - 29,345",None,None
3,Balkum Pada,"Rs. 8,542 - 9,095/sq. ft.",-0.48%,See Trends,"Rs. 10,846 - 14,297","Rs. 13,728 - 19,508",-,None,None
4,Bhaskar Colony,"Rs. 13,940 - 15,555/sq. ft.",-,Not Available,-,-,-,72.824121,19.1145401


In [7]:
# Create dataframe with only needed columns

mumbai_neighborhoods = mumbaipropertyvalue[['locality name', 'Price Range', 'Latitude','Longitude']].dropna().drop_duplicates()

In [8]:
mumbai_neighborhoods.head(15)

,locality name,Price Range,Latitude,Longitude
0,Anand Nagar,"Rs. 7,735 - 8,628/sq. ft.",18.9665234,72.8118877
1,Ashok Nagar,"Rs. 8,840 - 10,242/sq. ft.",19.0529955,72.8797875821267
4,Bhaskar Colony,"Rs. 13,940 - 15,555/sq. ft.",19.1145401,72.824121
20,G B Road,"Rs. 7,522 - 8,670/sq. ft.",18.935097,72.8255707
22,Hari Om Nagar,"Rs. 13,260 - 13,812/sq. ft.",19.1759697,72.9713361
27,Kailash Nagar,"Rs. 9,095 - 9,818/sq. ft.",19.0383631,72.9190035
41,Kopri,"Rs. 9,562 - 11,220/sq. ft.",19.0890741,73.0118496
42,Laxmi Nagar,"Rs. 11,220 - 12,410/sq. ft.",19.0908037,72.9167981
43,Lokmanya Nagar,-,18.9387711,72.8353355
44,Lokpuram,-,18.9387711,72.8353355


In [9]:
# check size
mumbai_neighborhoods.shape

(558, 4)

In [10]:
#remove rows with missing values
mumbai_neighborhoods.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [11]:
mumbai_neighborhoods

,locality name,Price Range,Latitude,Longitude
0,Anand Nagar,"Rs. 7,735 - 8,628/sq. ft.",18.9665234,72.8118877
1,Ashok Nagar,"Rs. 8,840 - 10,242/sq. ft.",19.0529955,72.8797875821267
4,Bhaskar Colony,"Rs. 13,940 - 15,555/sq. ft.",19.1145401,72.824121
20,G B Road,"Rs. 7,522 - 8,670/sq. ft.",18.935097,72.8255707
22,Hari Om Nagar,"Rs. 13,260 - 13,812/sq. ft.",19.1759697,72.9713361
27,Kailash Nagar,"Rs. 9,095 - 9,818/sq. ft.",19.0383631,72.9190035
41,Kopri,"Rs. 9,562 - 11,220/sq. ft.",19.0890741,73.0118496
42,Laxmi Nagar,"Rs. 11,220 - 12,410/sq. ft.",19.0908037,72.9167981
43,Lokmanya Nagar,-,18.9387711,72.8353355
44,Lokpuram,-,18.9387711,72.8353355


In [12]:
# check type of column Price Range
mumbai_neighborhoods['Price Range'].dtypes

dtype('O')

 <span style='color:Blue'>
    2b) Convert price range column to an integer that is an average of the two numbers in the range

In [13]:
# rename column
df_new = mumbai_neighborhoods.rename(columns={'Price Range': 'price'})

In [14]:
df_new.head()

,locality name,price,Latitude,Longitude
0,Anand Nagar,"Rs. 7,735 - 8,628/sq. ft.",18.9665234,72.8118877
1,Ashok Nagar,"Rs. 8,840 - 10,242/sq. ft.",19.0529955,72.8797875821267
4,Bhaskar Colony,"Rs. 13,940 - 15,555/sq. ft.",19.1145401,72.824121
20,G B Road,"Rs. 7,522 - 8,670/sq. ft.",18.935097,72.8255707
22,Hari Om Nagar,"Rs. 13,260 - 13,812/sq. ft.",19.1759697,72.9713361


In [15]:
# remove rows without Price Range value (marked with "-")
df_newer = df_new[df_new.price != '-']

In [16]:
# check that values were removed
df_newer.head(50)

,locality name,price,Latitude,Longitude
0,Anand Nagar,"Rs. 7,735 - 8,628/sq. ft.",18.9665234,72.8118877
1,Ashok Nagar,"Rs. 8,840 - 10,242/sq. ft.",19.0529955,72.8797875821267
4,Bhaskar Colony,"Rs. 13,940 - 15,555/sq. ft.",19.1145401,72.824121
20,G B Road,"Rs. 7,522 - 8,670/sq. ft.",18.935097,72.8255707
22,Hari Om Nagar,"Rs. 13,260 - 13,812/sq. ft.",19.1759697,72.9713361
27,Kailash Nagar,"Rs. 9,095 - 9,818/sq. ft.",19.0383631,72.9190035
41,Kopri,"Rs. 9,562 - 11,220/sq. ft.",19.0890741,73.0118496
42,Laxmi Nagar,"Rs. 11,220 - 12,410/sq. ft.",19.0908037,72.9167981
46,Louis Wadi,"Rs. 10,922 - 12,665/sq. ft.",19.1259981,72.8251458
51,Naupada,"Rs. 12,920 - 14,535/sq. ft.",19.083579,72.8886519


In [17]:
# check size
df_newer.shape

(478, 4)

In [18]:
# change price to number range only (by removing unnecessary characters)
df_newer["price"] = df_newer.price.str.replace("[Rs. , /sq. ft.]", "")

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [19]:
df_newer

,locality name,price,Latitude,Longitude
0,Anand Nagar,7735-8628,18.9665234,72.8118877
1,Ashok Nagar,8840-10242,19.0529955,72.8797875821267
4,Bhaskar Colony,13940-15555,19.1145401,72.824121
20,G B Road,7522-8670,18.935097,72.8255707
22,Hari Om Nagar,13260-13812,19.1759697,72.9713361
27,Kailash Nagar,9095-9818,19.0383631,72.9190035
41,Kopri,9562-11220,19.0890741,73.0118496
42,Laxmi Nagar,11220-12410,19.0908037,72.9167981
46,Louis Wadi,10922-12665,19.1259981,72.8251458
51,Naupada,12920-14535,19.083579,72.8886519


In [20]:
df_newer["price"] = df_newer.price.str.replace("[-]", " ")

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
df_newer

,locality name,price,Latitude,Longitude
0,Anand Nagar,7735 8628,18.9665234,72.8118877
1,Ashok Nagar,8840 10242,19.0529955,72.8797875821267
4,Bhaskar Colony,13940 15555,19.1145401,72.824121
20,G B Road,7522 8670,18.935097,72.8255707
22,Hari Om Nagar,13260 13812,19.1759697,72.9713361
27,Kailash Nagar,9095 9818,19.0383631,72.9190035
41,Kopri,9562 11220,19.0890741,73.0118496
42,Laxmi Nagar,11220 12410,19.0908037,72.9167981
46,Louis Wadi,10922 12665,19.1259981,72.8251458
51,Naupada,12920 14535,19.083579,72.8886519


In [22]:
# new data frame with split value columns 
new = df_newer["price"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
df_newer["price1"]= new[0] 
  
# making separate last name column from new data frame 
df_newer["price2"]= new[1] 
  
# Dropping old Name columns 
df_newer.drop(columns =["price"], inplace = True) 
  
# df display 
df_newer

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors

,locality name,Latitude,Longitude,price1,price2
0,Anand Nagar,18.9665234,72.8118877,7735,8628
1,Ashok Nagar,19.0529955,72.8797875821267,8840,10242
4,Bhaskar Colony,19.1145401,72.824121,13940,15555
20,G B Road,18.935097,72.8255707,7522,8670
22,Hari Om Nagar,19.1759697,72.9713361,13260,13812
27,Kailash Nagar,19.0383631,72.9190035,9095,9818
41,Kopri,19.0890741,73.0118496,9562,11220
42,Laxmi Nagar,19.0908037,72.9167981,11220,12410
46,Louis Wadi,19.1259981,72.8251458,10922,12665
51,Naupada,19.083579,72.8886519,12920,14535


In [23]:
# convert price1 and price 2 to integer datatype
df_newer["price1"] = df_newer["price1"].astype(str).astype(int)
df_newer["price2"] = df_newer["price2"].astype(str).astype(int)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [24]:
#average price1 and price2
df_newer['average_property_value'] = df_newer['price1'] + df_newer['price2']
df_newer

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,locality name,Latitude,Longitude,price1,price2,average_property_value
0,Anand Nagar,18.9665234,72.8118877,7735,8628,16363
1,Ashok Nagar,19.0529955,72.8797875821267,8840,10242,19082
4,Bhaskar Colony,19.1145401,72.824121,13940,15555,29495
20,G B Road,18.935097,72.8255707,7522,8670,16192
22,Hari Om Nagar,19.1759697,72.9713361,13260,13812,27072
27,Kailash Nagar,19.0383631,72.9190035,9095,9818,18913
41,Kopri,19.0890741,73.0118496,9562,11220,20782
42,Laxmi Nagar,19.0908037,72.9167981,11220,12410,23630
46,Louis Wadi,19.1259981,72.8251458,10922,12665,23587
51,Naupada,19.083579,72.8886519,12920,14535,27455


In [25]:
df_newer['average_property_value'] = (df_newer['average_property_value'])/2

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [26]:
df_newer.drop(columns=["price1", "price2"], inplace=True)
df_newer

,locality name,Latitude,Longitude,average_property_value
0,Anand Nagar,18.9665234,72.8118877,8181.5
1,Ashok Nagar,19.0529955,72.8797875821267,9541.0
4,Bhaskar Colony,19.1145401,72.824121,14747.5
20,G B Road,18.935097,72.8255707,8096.0
22,Hari Om Nagar,19.1759697,72.9713361,13536.0
27,Kailash Nagar,19.0383631,72.9190035,9456.5
41,Kopri,19.0890741,73.0118496,10391.0
42,Laxmi Nagar,19.0908037,72.9167981,11815.0
46,Louis Wadi,19.1259981,72.8251458,11793.5
51,Naupada,19.083579,72.8886519,13727.5


In [27]:
df_mumbai_neighborhoods = df_newer

In [28]:
# Now this is the dataframe that will be used for clustering and analysis (Finally all cleaned up and ready to use!)
df_mumbai_neighborhoods

,locality name,Latitude,Longitude,average_property_value
0,Anand Nagar,18.9665234,72.8118877,8181.5
1,Ashok Nagar,19.0529955,72.8797875821267,9541.0
4,Bhaskar Colony,19.1145401,72.824121,14747.5
20,G B Road,18.935097,72.8255707,8096.0
22,Hari Om Nagar,19.1759697,72.9713361,13536.0
27,Kailash Nagar,19.0383631,72.9190035,9456.5
41,Kopri,19.0890741,73.0118496,10391.0
42,Laxmi Nagar,19.0908037,72.9167981,11815.0
46,Louis Wadi,19.1259981,72.8251458,11793.5
51,Naupada,19.083579,72.8886519,13727.5


 <span style='color:Blue'>
    3) Create map of Mumbai and visualize neighborhood data

In [29]:
# convert latitude and longitude to float datatype

df_mumbai_neighborhoods["Latitude"] = df_mumbai_neighborhoods["Latitude"].astype(str).astype(float)
df_mumbai_neighborhoods["Longitude"] = df_mumbai_neighborhoods["Longitude"].astype(str).astype(float)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
# Set the mean latitude and longitude for Mumbai

latitude =df_mumbai_neighborhoods.Latitude.mean()
longitude =df_mumbai_neighborhoods.Longitude.mean()
print(latitude,longitude)

19.085631607845176 72.93419175280319


In [31]:
# Create map of Mumbai
mumbai_map = folium.Map(location=[latitude, longitude], zoom_start=10)


# add coordinates to the map as red circle markers
for lat, lng, label in zip(df_mumbai_neighborhoods.Latitude, df_mumbai_neighborhoods.Longitude, df_mumbai_neighborhoods['locality name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.7
        ).add_to(mumbai_map)

In [32]:
mumbai_map

In [33]:
df_mumbai_neighborhoods.reset_index(inplace = True)
df_mumbai_neighborhoods

,index,locality name,Latitude,Longitude,average_property_value
0,0,Anand Nagar,18.966523,72.811888,8181.5
1,1,Ashok Nagar,19.052996,72.879788,9541.0
2,4,Bhaskar Colony,19.114540,72.824121,14747.5
3,20,G B Road,18.935097,72.825571,8096.0
4,22,Hari Om Nagar,19.175970,72.971336,13536.0
5,27,Kailash Nagar,19.038363,72.919004,9456.5
6,41,Kopri,19.089074,73.011850,10391.0
7,42,Laxmi Nagar,19.090804,72.916798,11815.0
8,46,Louis Wadi,19.125998,72.825146,11793.5
9,51,Naupada,19.083579,72.888652,13727.5


In [34]:
df_mumbai_neighborhoods.drop(columns=['index'])

,locality name,Latitude,Longitude,average_property_value
0,Anand Nagar,18.966523,72.811888,8181.5
1,Ashok Nagar,19.052996,72.879788,9541.0
2,Bhaskar Colony,19.114540,72.824121,14747.5
3,G B Road,18.935097,72.825571,8096.0
4,Hari Om Nagar,19.175970,72.971336,13536.0
5,Kailash Nagar,19.038363,72.919004,9456.5
6,Kopri,19.089074,73.011850,10391.0
7,Laxmi Nagar,19.090804,72.916798,11815.0
8,Louis Wadi,19.125998,72.825146,11793.5
9,Naupada,19.083579,72.888652,13727.5


In [35]:
# remove outliers from dataset

df_mumbai_neighborhoods.drop(df_mumbai_neighborhoods.index[[154, 56, 20, 65, 66, 67, 32, 31, 35, 268, 279, 261, 233]], inplace=True)

In [36]:
# recreate map to ensure outliers are gone

# Create map of Mumbai
mumbai_map2 = folium.Map(location=[latitude, longitude], zoom_start=10)


# add coordinates to the map as red circle markers
for lat, lng, label in zip(df_mumbai_neighborhoods.Latitude, df_mumbai_neighborhoods.Longitude, df_mumbai_neighborhoods['locality name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.7
        ).add_to(mumbai_map2)

In [37]:
mumbai_map2

In [ ]:
# now the dataset 'df_mumbai_neighborhoods' and the map 'mumbai_map2' are prepared to be used for clustering analys

<span style='color:Blue'>
    4) Connect to Foursquare and use their data to find information on location of health care facilities in Mumbai

In [38]:
# Define Foursquare credentials and version
CLIENT_ID = 'TAGR1K5T2BYLKBE4KQAJAOFL1QSASUYDI00HP42OTLQXAYJN' # your Foursquare ID
CLIENT_SECRET = 'GW4ZDHDZGEFFJ2SAHATATI2MKFI3AXXTYUHZGIHMQ3XER3ZH' # your Foursquare Secret
VERSION = '20200424'
LIMIT = 100
radius = 15000
categoryId = '4bf58dd8d48988d104941735' # Medical Center category from https://developer.foursquare.com/docs/build-with-foursquare/categories/
latitude = '19.086849402258046'
longitude = '72.91617308349933'
intent = 'browse'

In [39]:
# Define the URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url

'https://api.foursquare.com/v2/venues/search?client_id=TAGR1K5T2BYLKBE4KQAJAOFL1QSASUYDI00HP42OTLQXAYJN&client_secret=GW4ZDHDZGEFFJ2SAHATATI2MKFI3AXXTYUHZGIHMQ3XER3ZH&ll=19.086849402258046,72.91617308349933&v=20200424&radius=15000&limit=100&categoryId=4bf58dd8d48988d104941735&intent=browse'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edd1bed71c428001bcfd411'},
 'response': {'venues': [{'id': '57e9db7c498ee2339fa6f27e',
    'name': 'SRL Diagnostics, SV Road',
    'location': {'address': 'SV Road',
     'lat': 19.171438,
     'lng': 72.84571,
     'labeledLatLngs': [{'label': 'display',
       'lat': 19.171438,
       'lng': 72.84571}],
     'distance': 11982,
     'postalCode': '400104',
     'cc': 'IN',
     'city': 'Mumbai',
     'state': 'Mahārāshtra',
     'country': 'India',
     'formattedAddress': ['SV Road', 'Mumbai 400104', 'Mahārāshtra', 'India']},
    'categories': [{'id': '4bf58dd8d48988d104941735',
      'name': 'Medical Center',
      'pluralName': 'Medical Centers',
      'shortName': 'Medical',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591549258',
    'hasPerk': False},
   {'id': '4e047336d4c06844661a0cd7',
    'name': 'Nanavati Hospital',
    'loca

In [41]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",False,57e9db7c498ee2339fa6f27e,SV Road,IN,Mumbai,India,NaN,11982,"[SV Road, Mumbai 400104, Mahārāshtra, India]","[{'label': 'display', 'lat': 19.171438, 'lng':...",19.171438,72.845710,NaN,400104,Mahārāshtra,"SRL Diagnostics, SV Road",v-1591549258
1,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4e047336d4c06844661a0cd7,SV Road,IN,Mumbai,India,Vile Parle,8084,"[SV Road (Vile Parle), Mumbai, Mahārāshtra, In...","[{'label': 'display', 'lat': 19.09588733246062...",19.095887,72.839919,NaN,NaN,Mahārāshtra,Nanavati Hospital,v-1591549258
2,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,54426af5498e12eec6601515,Next To hotel imperial palace,IN,Mumbai,India,Teli Gali Andheri East,7729,[Next To hotel imperial palace (Teli Gali Andh...,"[{'label': 'display', 'lat': 19.11826282422539...",19.118263,72.850639,NaN,NaN,Mahārāshtra,criticare hospital,v-1591549258
3,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,50fa8e5be4b0687cd4acef8b,NaN,IN,Mumbai,India,NaN,14553,"[Mumbai, Mahārāshtra, India]","[{'label': 'display', 'lat': 18.98985290527343...",18.989853,72.823441,NaN,NaN,Mahārāshtra,King George Memorial Hospital,v-1591549258
4,"[{'id': '4bf58dd8d48988d177941735', 'name': 'D...",False,5090a001e4b014dc94669d2d,Ground Floor @ Surya Sadan,IN,Mumbai,India,"Wadala-Sewree Road, Wadala North",9424,[Ground Floor @ Surya Sadan (Wadala-Sewree Roa...,"[{'label': 'display', 'lat': 19.0211685348769,...",19.021169,72.859663,NaN,400031,Mahārāshtra,Krishnaswamy's Clinic,v-1591549258
5,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4ebdea592c5b252c18c8c64f,NaN,IN,NaN,India,NaN,8786,[India],"[{'label': 'display', 'lat': 19.10724917540568...",19.107249,72.835488,NaN,NaN,NaN,Dr R.N.Cooper Mun.Gen Hospital,v-1591549258
6,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,526783f911d293604b0a32e2,NaN,IN,Mumbai,India,NaN,8237,"[Mumbai, Mahārāshtra, India]","[{'label': 'display', 'lat': 19.10781975682920...",19.107820,72.841079,NaN,NaN,Mahārāshtra,Kls Hospital,v-1591549258
7,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4b0587e4f964a5200ea622e3,"Hill Side Avenue, Hiranandani Gardens",IN,Mumbai,India,Powai,3733,"[Hill Side Avenue, Hiranandani Gardens (Powai)...","[{'label': 'display', 'lat': 19.12038555445533...",19.120386,72.916633,NaN,400 076,Mahārāshtra,Dr. L H Hiranandani Hospital,v-1591549258
8,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,4b488696f964a520bf4e26e3,Dr. Ambedkar Road,IN,Sion West,India,NaN,8215,"[Dr. Ambedkar Road, Sion West 400022, Mahārāsh...","[{'label': 'display', 'lat': 19.03596526794988...",19.035965,72.859622,NaN,400022,Mahārāshtra,Sion Hospital (LTMGH),v-1591549258
9,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",False,53240d3f498e9fe3fdbd1167,NaN,IN,NaN,India,NaN,10501,[India],"[{'label': 'display', 'lat': 19.01979239159748...",19.019792,72.845972,NaN,NaN,NaN,Navneet Jain Health Centre,v-1591549258


In [42]:
#create dataframe with only needed columns

mumbai_medical_venues = dataframe[['name', 'location.lat', 'location.lng']]
mumbai_medical_venues

,name,location.lat,location.lng
0,"SRL Diagnostics, SV Road",19.171438,72.845710
1,Nanavati Hospital,19.095887,72.839919
2,criticare hospital,19.118263,72.850639
3,King George Memorial Hospital,18.989853,72.823441
4,Krishnaswamy's Clinic,19.021169,72.859663
5,Dr R.N.Cooper Mun.Gen Hospital,19.107249,72.835488
6,Kls Hospital,19.107820,72.841079
7,Dr. L H Hiranandani Hospital,19.120386,72.916633
8,Sion Hospital (LTMGH),19.035965,72.859622
9,Navneet Jain Health Centre,19.019792,72.845972


In [43]:
#rename columns

mumbai_medical_venues.rename(columns={'name': 'venue_name', 'location.lat': 'venue_latitude', 'location.lng': 'venue_longitude'}, inplace=True)
mumbai_medical_venues.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,venue_name,venue_latitude,venue_longitude
0,"SRL Diagnostics, SV Road",19.171438,72.845710
1,Nanavati Hospital,19.095887,72.839919
2,criticare hospital,19.118263,72.850639
3,King George Memorial Hospital,18.989853,72.823441
4,Krishnaswamy's Clinic,19.021169,72.859663


In [44]:
# Map location of healthcare facilities in Mumbai

latitude = 19.086849402258046
longitude = 72.91617308349933

# Create map of Mumbai
mumbai_map3 = folium.Map(location=[latitude, longitude], zoom_start=10)


# add coordinates of medical venues to the map as red circle markers
for lat, lng, label in zip(mumbai_medical_venues.venue_latitude, mumbai_medical_venues.venue_longitude, mumbai_medical_venues['venue_name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.7
        ).add_to(mumbai_map3)

In [45]:
mumbai_map3

In [ ]:
# Let's organize data so we can find number of nearby medical venues for each neighborhood

In [46]:
df_mumbai_neighborhoods

,index,locality name,Latitude,Longitude,average_property_value
0,0,Anand Nagar,18.966523,72.811888,8181.5
1,1,Ashok Nagar,19.052996,72.879788,9541.0
2,4,Bhaskar Colony,19.114540,72.824121,14747.5
3,20,G B Road,18.935097,72.825571,8096.0
4,22,Hari Om Nagar,19.175970,72.971336,13536.0
5,27,Kailash Nagar,19.038363,72.919004,9456.5
6,41,Kopri,19.089074,73.011850,10391.0
7,42,Laxmi Nagar,19.090804,72.916798,11815.0
8,46,Louis Wadi,19.125998,72.825146,11793.5
9,51,Naupada,19.083579,72.888652,13727.5


In [47]:
df_mumbai_neighborhoods.sort_values(by=['average_property_value'], inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [48]:
# let's take only the 50 lowest income neighborhoods

df_mumbai_neighborhoods.head(51)

,index,locality name,Latitude,Longitude,average_property_value
43,190,Boisar,19.209144,72.859183,3187.5
40,158,Sahyadri Nagar,19.037232,72.906763,3209.0
26,115,Jadhav Colony,19.026027,72.852453,3230.0
27,117,Kailash Colony,19.140888,72.831892,3570.0
51,218,Morya Nagar,19.140625,72.833570,3995.0
53,220,Naigaon (West),19.009539,72.847871,4037.5
44,196,Dongarpada,19.166573,72.797899,4123.0
19,83,Ambivali West,19.133119,72.825134,4292.5
25,113,Hanuman Nagar,19.008007,72.823616,4462.5
331,695,Sector-20 Taloja,19.061486,73.116139,4526.0


In [49]:
less = df_mumbai_neighborhoods['average_property_value'] <= 6693.5


df_mumbai_low_income = df_mumbai_neighborhoods[less]

In [50]:
df_mumbai_low_income

,index,locality name,Latitude,Longitude,average_property_value
43,190,Boisar,19.209144,72.859183,3187.5
40,158,Sahyadri Nagar,19.037232,72.906763,3209.0
26,115,Jadhav Colony,19.026027,72.852453,3230.0
27,117,Kailash Colony,19.140888,72.831892,3570.0
51,218,Morya Nagar,19.140625,72.833570,3995.0
53,220,Naigaon (West),19.009539,72.847871,4037.5
44,196,Dongarpada,19.166573,72.797899,4123.0
19,83,Ambivali West,19.133119,72.825134,4292.5
25,113,Hanuman Nagar,19.008007,72.823616,4462.5
331,695,Sector-20 Taloja,19.061486,73.116139,4526.0


In [51]:
df_mumbai_low_income.reset_index(inplace=True)

In [52]:
df_mumbai_low_income

,level_0,index,locality name,Latitude,Longitude,average_property_value
0,43,190,Boisar,19.209144,72.859183,3187.5
1,40,158,Sahyadri Nagar,19.037232,72.906763,3209.0
2,26,115,Jadhav Colony,19.026027,72.852453,3230.0
3,27,117,Kailash Colony,19.140888,72.831892,3570.0
4,51,218,Morya Nagar,19.140625,72.833570,3995.0
5,53,220,Naigaon (West),19.009539,72.847871,4037.5
6,44,196,Dongarpada,19.166573,72.797899,4123.0
7,19,83,Ambivali West,19.133119,72.825134,4292.5
8,25,113,Hanuman Nagar,19.008007,72.823616,4462.5
9,331,695,Sector-20 Taloja,19.061486,73.116139,4526.0


In [53]:
df_mumbai_low_income.drop(columns=['level_0', 'index'], inplace=True)

In [54]:
df_mumbai_low_income

,locality name,Latitude,Longitude,average_property_value
0,Boisar,19.209144,72.859183,3187.5
1,Sahyadri Nagar,19.037232,72.906763,3209.0
2,Jadhav Colony,19.026027,72.852453,3230.0
3,Kailash Colony,19.140888,72.831892,3570.0
4,Morya Nagar,19.140625,72.833570,3995.0
5,Naigaon (West),19.009539,72.847871,4037.5
6,Dongarpada,19.166573,72.797899,4123.0
7,Ambivali West,19.133119,72.825134,4292.5
8,Hanuman Nagar,19.008007,72.823616,4462.5
9,Sector-20 Taloja,19.061486,73.116139,4526.0


In [55]:
# Manually check to see how many nearby medical venues come up for each neighborhood by chaning URL lat and long each time

In [56]:
#1 Boisar

# Define Foursquare credentials and version
CLIENT_ID = 'TAGR1K5T2BYLKBE4KQAJAOFL1QSASUYDI00HP42OTLQXAYJN' # your Foursquare ID
CLIENT_SECRET = 'GW4ZDHDZGEFFJ2SAHATATI2MKFI3AXXTYUHZGIHMQ3XER3ZH' # your Foursquare Secret
VERSION = '20200424'
LIMIT = 100
radius = 500
categoryId = '4bf58dd8d48988d104941735' # Medical Center category from https://developer.foursquare.com/docs/build-with-foursquare/categories/
latitude = 19.209144
longitude = 72.859183
intent = 'browse'

# Define the URL
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url1

# Make the call
results1 = requests.get(url1).json()

# assign relevant part of JSON to medical_venues
medical_venues1 = results1['response']['venues']

# tranform venues into a dataframe
dataframe1 = json_normalize(medical_venues1)

print('{} venues were returned by Foursquare.'.format(dataframe1.shape[0]))

5 venues were returned by Foursquare.


In [57]:
#2 Sahyadri Nagar

# Define Foursquare credentials

latitude = 19.037232
longitude = 72.906763

# Define the URL
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url2

# Make the call
results2 = requests.get(url2).json()

# assign relevant part of JSON to medical_venues
medical_venues2 = results2['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(medical_venues2)

print('{} venues were returned by Foursquare.'.format(dataframe2.shape[0]))

0 venues were returned by Foursquare.


In [58]:
#3 Jadhav Colony

# Define Foursquare credentials

latitude = 19.026027
longitude = 72.852453

# Define the URL
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url3

# Make the call
results3 = requests.get(url3).json()

# assign relevant part of JSON to medical_venues
medical_venues3 = results3['response']['venues']

# tranform venues into a dataframe
dataframe3 = json_normalize(medical_venues3)

print('{} venues were returned by Foursquare.'.format(dataframe3.shape[0]))

31 venues were returned by Foursquare.


In [59]:
#4 Kailash Colony

# Define Foursquare credentials

latitude = 19.140888
longitude = 72.831892

# Define the URL
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url4

# Make the call
results4 = requests.get(url4).json()

# assign relevant part of JSON to medical_venues
medical_venues4 = results4['response']['venues']

# tranform venues into a dataframe
dataframe4 = json_normalize(medical_venues4)

print('{} venues were returned by Foursquare.'.format(dataframe4.shape[0]))

12 venues were returned by Foursquare.


In [60]:
#5 Morya Nagar

# Define Foursquare credentials

latitude = 19.140625
longitude = 72.833570

# Define the URL
url5 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url5

# Make the call
results5 = requests.get(url5).json()

# assign relevant part of JSON to medical_venues
medical_venues5 = results5['response']['venues']

# tranform venues into a dataframe
dataframe5 = json_normalize(medical_venues5)

print('{} venues were returned by Foursquare.'.format(dataframe5.shape[0]))

12 venues were returned by Foursquare.


In [61]:
#6 Naigaon (West)

# Define Foursquare credentials

latitude = 19.009539
longitude = 72.847871

# Define the URL
url6 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url6

# Make the call
results6 = requests.get(url6).json()

# assign relevant part of JSON to medical_venues
medical_venues6 = results6['response']['venues']

# tranform venues into a dataframe
dataframe6 = json_normalize(medical_venues6)

print('{} venues were returned by Foursquare.'.format(dataframe6.shape[0]))

5 venues were returned by Foursquare.


In [62]:
#7 Dongarpada

# Define Foursquare credentials

latitude = 19.166573
longitude = 72.797899

# Define the URL
url7 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url7

# Make the call
results7 = requests.get(url7).json()

# assign relevant part of JSON to medical_venues
medical_venues7 = results7['response']['venues']

# tranform venues into a dataframe
dataframe7 = json_normalize(medical_venues7)

print('{} venues were returned by Foursquare.'.format(dataframe7.shape[0]))

0 venues were returned by Foursquare.


In [63]:
#8 Ambivali West

# Define Foursquare credentials

latitude = 19.133119
longitude = 72.825134

# Define the URL
url8 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url8

# Make the call
results8 = requests.get(url8).json()

# assign relevant part of JSON to medical_venues
medical_venues8 = results8['response']['venues']

# tranform venues into a dataframe
dataframe8 = json_normalize(medical_venues8)

print('{} venues were returned by Foursquare.'.format(dataframe8.shape[0]))

40 venues were returned by Foursquare.


In [64]:
#9 Hanuman Nagar

# Define Foursquare credentials

latitude = 19.008007
longitude = 72.823616

# Define the URL
url9 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url9

# Make the call
results9 = requests.get(url9).json()

# assign relevant part of JSON to medical_venues
medical_venues9 = results9['response']['venues']

# tranform venues into a dataframe
dataframe9 = json_normalize(medical_venues9)

print('{} venues were returned by Foursquare.'.format(dataframe9.shape[0]))

4 venues were returned by Foursquare.


In [65]:
#10 Sector-20 Taloja

# Define Foursquare credentials

latitude = 19.061486
longitude = 73.116139

# Define the URL
url10 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url10

# Make the call
results10 = requests.get(url10).json()

# assign relevant part of JSON to medical_venues
medical_venues10 = results10['response']['venues']

# tranform venues into a dataframe
dataframe10 = json_normalize(medical_venues10)

print('{} venues were returned by Foursquare.'.format(dataframe10.shape[0]))

0 venues were returned by Foursquare.


In [66]:
#11 Sector-26 Taloja

# Define Foursquare credentials

latitude = 19.061486
longitude = 73.116139

# Define the URL
url11 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url11

# Make the call
results11 = requests.get(url11).json()

# assign relevant part of JSON to medical_venues
medical_venues11 = results11['response']['venues']

# tranform venues into a dataframe
dataframe11 = json_normalize(medical_venues11)

print('{} venues were returned by Foursquare.'.format(dataframe11.shape[0]))

0 venues were returned by Foursquare.


In [67]:
#12 New Panvel East

# Define Foursquare credentials

latitude = 18.938771
longitude = 72.835335

# Define the URL
url12 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url12

# Make the call
results12 = requests.get(url12).json()

# assign relevant part of JSON to medical_venues
medical_venues12 = results12['response']['venues']

# tranform venues into a dataframe
dataframe12 = json_normalize(medical_venues12)

print('{} venues were returned by Foursquare.'.format(dataframe12.shape[0]))

12 venues were returned by Foursquare.


In [68]:
#13 Sector-11 Taloja

# Define Foursquare credentials

latitude = 19.061486
longitude = 73.116139

# Define the URL
url13 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url13

# Make the call
results13 = requests.get(url13).json()

# assign relevant part of JSON to medical_venues
medical_venues13 = results13['response']['venues']

# tranform venues into a dataframe
dataframe13 = json_normalize(medical_venues13)

print('{} venues were returned by Foursquare.'.format(dataframe13.shape[0]))

0 venues were returned by Foursquare.


In [69]:
#14 Kalyan (East)

# Define Foursquare credentials

latitude = 19.137892
longitude = 72.810668

# Define the URL
url14 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url14

# Make the call
results14 = requests.get(url14).json()

# assign relevant part of JSON to medical_venues
medical_venues14 = results14['response']['venues']

# tranform venues into a dataframe
dataframe14 = json_normalize(medical_venues14)

print('{} venues were returned by Foursquare.'.format(dataframe14.shape[0]))

13 venues were returned by Foursquare.


In [70]:
#15 Akurli

# Define Foursquare credentials

latitude = 19.200274
longitude = 72.876951

# Define the URL
url15 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url15

# Make the call
results15 = requests.get(url15).json()

# assign relevant part of JSON to medical_venues
medical_venues15 = results15['response']['venues']

# tranform venues into a dataframe
dataframe15 = json_normalize(medical_venues15)

print('{} venues were returned by Foursquare.'.format(dataframe15.shape[0]))

1 venues were returned by Foursquare.


In [71]:
#16 Naigaon (East)

# Define Foursquare credentials

latitude = 19.009539
longitude = 72.847871

# Define the URL
url16 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url16

# Make the call
results16 = requests.get(url16).json()

# assign relevant part of JSON to medical_venues
medical_venues16 = results16['response']['venues']

# tranform venues into a dataframe
dataframe16 = json_normalize(medical_venues16)

print('{} venues were returned by Foursquare.'.format(dataframe16.shape[0]))

5 venues were returned by Foursquare.


In [72]:
#17 Taloja

# Define Foursquare credentials

latitude = 19.061486
longitude = 73.116139

# Define the URL
url17 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url17

# Make the call
results17 = requests.get(url17).json()

# assign relevant part of JSON to medical_venues
medical_venues17 = results17['response']['venues']

# tranform venues into a dataframe
dataframe17 = json_normalize(medical_venues17)

print('{} venues were returned by Foursquare.'.format(dataframe17.shape[0]))

0 venues were returned by Foursquare.


In [73]:
#18 Navade

# Define Foursquare credentials

latitude = 19.053846
longitude = 73.103030

# Define the URL
url18 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url18

# Make the call
results18 = requests.get(url18).json()

# assign relevant part of JSON to medical_venues
medical_venues18 = results18['response']['venues']

# tranform venues into a dataframe
dataframe18 = json_normalize(medical_venues18)

print('{} venues were returned by Foursquare.'.format(dataframe18.shape[0]))

0 venues were returned by Foursquare.


In [74]:
#19 Sector-16 Taloja

# Define Foursquare credentials

latitude = 19.061486
longitude = 73.116139

# Define the URL
url19 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url19

# Make the call
results19 = requests.get(url19).json()

# assign relevant part of JSON to medical_venues
medical_venues19 = results19['response']['venues']

# tranform venues into a dataframe
dataframe19 = json_normalize(medical_venues19)

print('{} venues were returned by Foursquare.'.format(dataframe19.shape[0]))

0 venues were returned by Foursquare.


In [75]:
#20 Pragati Nagar

# Define Foursquare credentials

latitude = 19.058373
longitude = 72.888755

# Define the URL
url20 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url20

# Make the call
results20 = requests.get(url20).json()

# assign relevant part of JSON to medical_venues
medical_venues20 = results20['response']['venues']

# tranform venues into a dataframe
dataframe20 = json_normalize(medical_venues20)

print('{} venues were returned by Foursquare.'.format(dataframe20.shape[0]))

4 venues were returned by Foursquare.


In [76]:
#21 Chanakya Nagar

# Define Foursquare credentials

latitude = 19.204654
longitude = 72.854837

# Define the URL
url21 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url21

# Make the call
results21 = requests.get(url21).json()

# assign relevant part of JSON to medical_venues
medical_venues21 = results21['response']['venues']

# tranform venues into a dataframe
dataframe21 = json_normalize(medical_venues21)

print('{} venues were returned by Foursquare.'.format(dataframe21.shape[0]))

9 venues were returned by Foursquare.


In [77]:
#22 Y K Nagar			

# Define Foursquare credentials

latitude = 19.026298
longitude = 72.873033

# Define the URL
url22 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url22

# Make the call
results22 = requests.get(url22).json()

# assign relevant part of JSON to medical_venues
medical_venues22 = results22['response']['venues']

# tranform venues into a dataframe
dataframe22 = json_normalize(medical_venues22)

print('{} venues were returned by Foursquare.'.format(dataframe22.shape[0]))

0 venues were returned by Foursquare.


In [78]:
#23 Sector-14 Taloja		

# Define Foursquare credentials

latitude = 19.061486
longitude = 73.116139

# Define the URL
url23 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url23

# Make the call
results23 = requests.get(url23).json()

# assign relevant part of JSON to medical_venues
medical_venues23 = results23['response']['venues']

# tranform venues into a dataframe
dataframe23 = json_normalize(medical_venues23)

print('{} venues were returned by Foursquare.'.format(dataframe23.shape[0]))

0 venues were returned by Foursquare.


In [79]:
#24 Tirupati Nagar

# Define Foursquare credentials

latitude = 19.113605
longitude = 72.880480

# Define the URL
url24 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url24

# Make the call
results24 = requests.get(url24).json()

# assign relevant part of JSON to medical_venues
medical_venues24 = results24['response']['venues']

# tranform venues into a dataframe
dataframe24 = json_normalize(medical_venues24)

print('{} venues were returned by Foursquare.'.format(dataframe24.shape[0]))

11 venues were returned by Foursquare.


In [80]:
#25 P And T Colony

# Define Foursquare credentials

latitude = 19.101937
longitude = 72.861599

# Define the URL
url25 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url25

# Make the call
results25 = requests.get(url25).json()

# assign relevant part of JSON to medical_venues
medical_venues25 = results25['response']['venues']

# tranform venues into a dataframe
dataframe25 = json_normalize(medical_venues25)

print('{} venues were returned by Foursquare.'.format(dataframe25.shape[0]))

0 venues were returned by Foursquare.


In [81]:
#26 Sector-23 Taloja			

# Define Foursquare credentials

latitude = 19.061486
longitude = 73.116139

# Define the URL
url26 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url26

# Make the call
results26 = requests.get(url26).json()

# assign relevant part of JSON to medical_venues
medical_venues26 = results26['response']['venues']

# tranform venues into a dataframe
dataframe26 = json_normalize(medical_venues26)

print('{} venues were returned by Foursquare.'.format(dataframe26.shape[0]))

0 venues were returned by Foursquare.


In [82]:
#27 Sai Nagar

# Define Foursquare credentials

latitude = 19.149018
longitude = 72.925706

# Define the URL
url27 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url27

# Make the call
results27 = requests.get(url27).json()

# assign relevant part of JSON to medical_venues
medical_venues27 = results27['response']['venues']

# tranform venues into a dataframe
dataframe27 = json_normalize(medical_venues27)

print('{} venues were returned by Foursquare.'.format(dataframe27.shape[0]))

2 venues were returned by Foursquare.


In [83]:
#28 Kalyan (West)

# Define Foursquare credentials

latitude = 19.137892
longitude = 72.810668

# Define the URL
url28 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url28

# Make the call
results28 = requests.get(url28).json()

# assign relevant part of JSON to medical_venues
medical_venues28 = results28['response']['venues']

# tranform venues into a dataframe
dataframe28 = json_normalize(medical_venues28)

print('{} venues were returned by Foursquare.'.format(dataframe28.shape[0]))

13 venues were returned by Foursquare.


In [84]:
#29 M I D C

# Define Foursquare credentials

latitude = 19.121958
longitude = 72.909294

# Define the URL
url29 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url29

# Make the call
results29 = requests.get(url29).json()

# assign relevant part of JSON to medical_venues
medical_venues29 = results29['response']['venues']

# tranform venues into a dataframe
dataframe29 = json_normalize(medical_venues29)

print('{} venues were returned by Foursquare.'.format(dataframe29.shape[0]))

11 venues were returned by Foursquare.


In [85]:
#30 Sector-19 Taloja

# Define Foursquare credentials

latitude = 19.061486
longitude = 73.116139

# Define the URL
url30 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url30

# Make the call
results30 = requests.get(url30).json()

# assign relevant part of JSON to medical_venues
medical_venues30 = results30['response']['venues']

# tranform venues into a dataframe
dataframe30 = json_normalize(medical_venues30)

print('{} venues were returned by Foursquare.'.format(dataframe30.shape[0]))

0 venues were returned by Foursquare.


In [86]:
#31 Maharashtra Nagar

# Define Foursquare credentials

latitude = 19.055997
longitude = 72.922665

# Define the URL
url31 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url31

# Make the call
results31 = requests.get(url31).json()

# assign relevant part of JSON to medical_venues
medical_venues31 = results31['response']['venues']

# tranform venues into a dataframe
dataframe31 = json_normalize(medical_venues31)

print('{} venues were returned by Foursquare.'.format(dataframe31.shape[0]))

0 venues were returned by Foursquare.


In [87]:
#32 Gandhi Nagar

# Define Foursquare credentials

latitude = 19.058040
longitude = 72.847258

# Define the URL
url32 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url32

# Make the call
results32 = requests.get(url32).json()

# assign relevant part of JSON to medical_venues
medical_venues32 = results32['response']['venues']

# tranform venues into a dataframe
dataframe32 = json_normalize(medical_venues32)

print('{} venues were returned by Foursquare.'.format(dataframe32.shape[0]))

10 venues were returned by Foursquare.


In [88]:
#33 Radha Nagar

# Define Foursquare credentials

latitude = 19.026083
longitude = 72.855560

# Define the URL
url33 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url33

# Make the call
results33 = requests.get(url33).json()

# assign relevant part of JSON to medical_venues
medical_venues33 = results33['response']['venues']

# tranform venues into a dataframe
dataframe33 = json_normalize(medical_venues33)

print('{} venues were returned by Foursquare.'.format(dataframe33.shape[0]))

22 venues were returned by Foursquare.


In [89]:
#34 Sector-24 Taloja

# Define Foursquare credentials

latitude = 19.073701
longitude = 73.081935

# Define the URL
url34 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url34

# Make the call
results34 = requests.get(url34).json()

# assign relevant part of JSON to medical_venues
medical_venues34 = results34['response']['venues']

# tranform venues into a dataframe
dataframe34 = json_normalize(medical_venues34)

print('{} venues were returned by Foursquare.'.format(dataframe34.shape[0]))

0 venues were returned by Foursquare.


In [90]:
#35 Pendse Nagar

# Define Foursquare credentials

latitude = 19.165626
longitude = 72.852589

# Define the URL
url35 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url35

# Make the call
results35 = requests.get(url35).json()

# assign relevant part of JSON to medical_venues
medical_venues35 = results35['response']['venues']

# tranform venues into a dataframe
dataframe35 = json_normalize(medical_venues35)

print('{} venues were returned by Foursquare.'.format(dataframe35.shape[0]))

20 venues were returned by Foursquare.


In [91]:
#36 Ganesh Nagar

# Define Foursquare credentials

latitude = 19.151548
longitude = 72.930062

# Define the URL
url36 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url36

# Make the call
results36 = requests.get(url36).json()

# assign relevant part of JSON to medical_venues
medical_venues36 = results36['response']['venues']

# tranform venues into a dataframe
dataframe36 = json_normalize(medical_venues36)

print('{} venues were returned by Foursquare.'.format(dataframe36.shape[0]))

1 venues were returned by Foursquare.


In [92]:
#37 Kalu Nagar

# Define Foursquare credentials

latitude = 19.006303
longitude = 72.819127

# Define the URL
url37 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url37

# Make the call
results37 = requests.get(url37).json()

# assign relevant part of JSON to medical_venues
medical_venues37 = results37['response']['venues']

# tranform venues into a dataframe
dataframe37 = json_normalize(medical_venues37)

print('{} venues were returned by Foursquare.'.format(dataframe37.shape[0]))

3 venues were returned by Foursquare.


In [93]:
#38 Shastri Nagar

# Define Foursquare credentials

latitude = 19.138039
longitude = 72.828016

# Define the URL
url38 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url38

# Make the call
results38 = requests.get(url38).json()

# assign relevant part of JSON to medical_venues
medical_venues38 = results38['response']['venues']

# tranform venues into a dataframe
dataframe38 = json_normalize(medical_venues38)

print('{} venues were returned by Foursquare.'.format(dataframe38.shape[0]))

34 venues were returned by Foursquare.


In [94]:
#39 Suyog Nagar

# Define Foursquare credentials

latitude = 19.166501
longitude = 72.950308

# Define the URL
url39 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url39

# Make the call
results39 = requests.get(url39).json()

# assign relevant part of JSON to medical_venues
medical_venues39 = results39['response']['venues']

# tranform venues into a dataframe
dataframe39 = json_normalize(medical_venues39)

print('{} venues were returned by Foursquare.'.format(dataframe39.shape[0]))

8 venues were returned by Foursquare.


In [95]:
#40 Sector-10 Kalamboli

# Define Foursquare credentials

latitude = 19.023506
longitude = 73.110609

# Define the URL
url40 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url40

# Make the call
results40 = requests.get(url40).json()

# assign relevant part of JSON to medical_venues
medical_venues40 = results40['response']['venues']

# tranform venues into a dataframe
dataframe40 = json_normalize(medical_venues40)

print('{} venues were returned by Foursquare.'.format(dataframe40.shape[0]))

0 venues were returned by Foursquare.


In [96]:
#41 Sector 19 Ulwe

# Define Foursquare credentials

latitude = 18.980436
longitude = 73.038731

# Define the URL
url41 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url41

# Make the call
results41 = requests.get(url41).json()

# assign relevant part of JSON to medical_venues
medical_venues41 = results41['response']['venues']

# tranform venues into a dataframe
dataframe41 = json_normalize(medical_venues41)

print('{} venues were returned by Foursquare.'.format(dataframe41.shape[0]))

0 venues were returned by Foursquare.


In [97]:
#42 Sangeeta Wadi

# Define Foursquare credentials

latitude = 19.079566
longitude = 72.833485

# Define the URL
url42 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url42

# Make the call
results42 = requests.get(url42).json()

# assign relevant part of JSON to medical_venues
medical_venues42 = results42['response']['venues']

# tranform venues into a dataframe
dataframe42 = json_normalize(medical_venues42)

print('{} venues were returned by Foursquare.'.format(dataframe42.shape[0]))

25 venues were returned by Foursquare.


In [98]:
#43 Panvel

# Define Foursquare credentials

latitude = 19.050027
longitude = 72.933334

# Define the URL
url43 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url43

# Make the call
results43 = requests.get(url43).json()

# assign relevant part of JSON to medical_venues
medical_venues43 = results43['response']['venues']

# tranform venues into a dataframe
dataframe43 = json_normalize(medical_venues43)

print('{} venues were returned by Foursquare.'.format(dataframe43.shape[0]))

1 venues were returned by Foursquare.


In [99]:
#44 New Panvel

# Define Foursquare credentials

latitude = 18.938771
longitude = 72.835335

# Define the URL
url44 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url44

# Make the call
results44 = requests.get(url44).json()

# assign relevant part of JSON to medical_venues
medical_venues44 = results44['response']['venues']

# tranform venues into a dataframe
dataframe44 = json_normalize(medical_venues44)

print('{} venues were returned by Foursquare.'.format(dataframe44.shape[0]))

12 venues were returned by Foursquare.


In [100]:
#45 Sector 2 Ulwe			

# Define Foursquare credentials

latitude = 18.980436
longitude = 73.038731

# Define the URL
url45 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url45

# Make the call
results45 = requests.get(url45).json()

# assign relevant part of JSON to medical_venues
medical_venues45 = results45['response']['venues']

# tranform venues into a dataframe
dataframe45 = json_normalize(medical_venues45)

print('{} venues were returned by Foursquare.'.format(dataframe45.shape[0]))

0 venues were returned by Foursquare.


In [101]:
#46 Balaji Nagar

# Define Foursquare credentials

latitude = 19.061083
longitude = 72.900459

# Define the URL
url46 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url46

# Make the call
results46 = requests.get(url46).json()

# assign relevant part of JSON to medical_venues
medical_venues46 = results46['response']['venues']

# tranform venues into a dataframe
dataframe46 = json_normalize(medical_venues46)

print('{} venues were returned by Foursquare.'.format(dataframe46.shape[0]))

43 venues were returned by Foursquare.


In [102]:
#47 Old Panvel

# Define Foursquare credentials

latitude = 19.050027
longitude = 72.933334

# Define the URL
url47 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url47

# Make the call
results47 = requests.get(url47).json()

# assign relevant part of JSON to medical_venues
medical_venues47 = results47['response']['venues']

# tranform venues into a dataframe
dataframe47 = json_normalize(medical_venues47)

print('{} venues were returned by Foursquare.'.format(dataframe47.shape[0]))

1 venues were returned by Foursquare.


In [103]:
#48 Sector 30 Kharghar

# Define Foursquare credentials

latitude = 19.037734
longitude = 73.064192

# Define the URL
url48 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url48

# Make the call
results48 = requests.get(url48).json()

# assign relevant part of JSON to medical_venues
medical_venues48 = results48['response']['venues']

# tranform venues into a dataframe
dataframe48 = json_normalize(medical_venues48)

print('{} venues were returned by Foursquare.'.format(dataframe48.shape[0]))

7 venues were returned by Foursquare.


In [104]:
#49 Kalamboli

# Define Foursquare credentials

latitude = 19.023506
longitude = 73.110609

# Define the URL
url49 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url49

# Make the call
results49 = requests.get(url49).json()

# assign relevant part of JSON to medical_venues
medical_venues49 = results49['response']['venues']

# tranform venues into a dataframe
dataframe49 = json_normalize(medical_venues49)

print('{} venues were returned by Foursquare.'.format(dataframe49.shape[0]))

0 venues were returned by Foursquare.


In [105]:
#50 Sector-3 Ulwe

# Define Foursquare credentials

latitude = 18.980436
longitude = 73.038731

# Define the URL
url50 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url50

# Make the call
results50 = requests.get(url50).json()

# assign relevant part of JSON to medical_venues
medical_venues50 = results50['response']['venues']

# tranform venues into a dataframe
dataframe50 = json_normalize(medical_venues50)

print('{} venues were returned by Foursquare.'.format(dataframe50.shape[0]))

0 venues were returned by Foursquare.


In [117]:
#51 Sector-34 Kharghar

# Define Foursquare credentials

latitude = 19.073701
longitude = 73.081935

# Define the URL
url51 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}&intent={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT, categoryId, intent)
url51

# Make the call
results51 = requests.get(url51).json()

# assign relevant part of JSON to medical_venues
medical_venues51 = results51['response']['venues']

# tranform venues into a dataframe
dataframe51 = json_normalize(medical_venues51)

print('{} venues were returned by Foursquare.'.format(dataframe51.shape[0]))

0 venues were returned by Foursquare.


In [115]:
df_mumbai_low_income

,locality name,Latitude,Longitude,average_property_value
0,Boisar,19.209144,72.859183,3187.5
1,Sahyadri Nagar,19.037232,72.906763,3209.0
2,Jadhav Colony,19.026027,72.852453,3230.0
3,Kailash Colony,19.140888,72.831892,3570.0
4,Morya Nagar,19.140625,72.833570,3995.0
5,Naigaon (West),19.009539,72.847871,4037.5
6,Dongarpada,19.166573,72.797899,4123.0
7,Ambivali West,19.133119,72.825134,4292.5
8,Hanuman Nagar,19.008007,72.823616,4462.5
9,Sector-20 Taloja,19.061486,73.116139,4526.0


In [118]:
# Declare a list that is to be converted into a column 
number_of_venues = ['5', '0', '31', '12', '12', '5', '0', '40',
                    '4', '0', '0', '12', '0', '13', '1', '5', 
                    '0', '0', '0', '4', '9', '0', '0', '11', 
                    '0', '0', '2', '13', '11', '0', '0', '10', 
                    '22', '0', '20', '1', '3', '34', '8', '0', 
                    '0', '25', '1', '12', '0', '43', '1', '7', 
                    '0', '0', '0']

In [119]:
# Using 'number_of_venues' as the column name 
# and equating it to the list 
df_mumbai_low_income['number_of_venues'] = number_of_venues

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [120]:
df_mumbai_low_income

,locality name,Latitude,Longitude,average_property_value,number_of_venues
0,Boisar,19.209144,72.859183,3187.5,5
1,Sahyadri Nagar,19.037232,72.906763,3209.0,0
2,Jadhav Colony,19.026027,72.852453,3230.0,31
3,Kailash Colony,19.140888,72.831892,3570.0,12
4,Morya Nagar,19.140625,72.833570,3995.0,12
5,Naigaon (West),19.009539,72.847871,4037.5,5
6,Dongarpada,19.166573,72.797899,4123.0,0
7,Ambivali West,19.133119,72.825134,4292.5,40
8,Hanuman Nagar,19.008007,72.823616,4462.5,4
9,Sector-20 Taloja,19.061486,73.116139,4526.0,0


## Now we cluster

In [121]:
# Run k-means to cluster the neighborhood into 10 clusters

# set number of clusters
kclusters = 10

mumbai_clustering = df_mumbai_low_income.drop('locality name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 7, 7, 7, 7, 1, 1], dtype=int32)

In [122]:
# add clustering labels
df_mumbai_low_income.insert(0, 'Cluster Labels', kmeans.labels_)

In [123]:
df_mumbai_low_income

,Cluster Labels,locality name,Latitude,Longitude,average_property_value,number_of_venues
0,3,Boisar,19.209144,72.859183,3187.5,5
1,3,Sahyadri Nagar,19.037232,72.906763,3209.0,0
2,3,Jadhav Colony,19.026027,72.852453,3230.0,31
3,3,Kailash Colony,19.140888,72.831892,3570.0,12
4,7,Morya Nagar,19.140625,72.833570,3995.0,12
5,7,Naigaon (West),19.009539,72.847871,4037.5,5
6,7,Dongarpada,19.166573,72.797899,4123.0,0
7,7,Ambivali West,19.133119,72.825134,4292.5,40
8,1,Hanuman Nagar,19.008007,72.823616,4462.5,4
9,1,Sector-20 Taloja,19.061486,73.116139,4526.0,0


In [125]:
# create map
map_clusters = folium.Map(location=[19.085631607845176, 72.93419175280319], zoom_start=10)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_mumbai_low_income['Latitude'], 
                                  df_mumbai_low_income['Longitude'], 
                                  df_mumbai_low_income['locality name'], 
                                  df_mumbai_low_income['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [126]:
map_clusters

In [127]:
# try with 5 clusters

# set number of clusters
kclusters = 5

mumbai_clustering2 = df_mumbai_low_income.drop('locality name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering2)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 2, 2, 2, 2, 2, 2], dtype=int32)

In [128]:
# add cluster labels
df_mumbai_low_income.insert(0, 'Cluster Label', kmeans.labels_)

In [129]:
df_mumbai_low_income

,Cluster Label,Cluster Labels,locality name,Latitude,Longitude,average_property_value,number_of_venues
0,4,3,Boisar,19.209144,72.859183,3187.5,5
1,4,3,Sahyadri Nagar,19.037232,72.906763,3209.0,0
2,4,3,Jadhav Colony,19.026027,72.852453,3230.0,31
3,4,3,Kailash Colony,19.140888,72.831892,3570.0,12
4,2,7,Morya Nagar,19.140625,72.833570,3995.0,12
5,2,7,Naigaon (West),19.009539,72.847871,4037.5,5
6,2,7,Dongarpada,19.166573,72.797899,4123.0,0
7,2,7,Ambivali West,19.133119,72.825134,4292.5,40
8,2,1,Hanuman Nagar,19.008007,72.823616,4462.5,4
9,2,1,Sector-20 Taloja,19.061486,73.116139,4526.0,0


In [130]:
# create map with new clustering

map_clusters2 = folium.Map(location=[19.085631607845176, 72.93419175280319], zoom_start=10)


# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_mumbai_low_income['Latitude'], 
                                  df_mumbai_low_income['Longitude'], 
                                  df_mumbai_low_income['locality name'], 
                                  df_mumbai_low_income['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters2)

In [131]:
map_clusters2

# FINAL CONCLUSION:

### In the second map, you can see that cluster 3 (light green color) is the cluster with the most geographical consistency and according to the table a low average number of nearby medical venues. Therefore, I would suggest building the new medical clinic somewhere in the middle of these neighborhoods to best serve the community.

#### HOWEVER, we can clearly see that the clusters are not very uniform, so further analysis or a different of clustering might be better suited to answer this very specific question.